# 파이썬 버전 3.9.6

### 아래 라이브러리를 설치해야합니다. '#'를 지우고  pip 마다 각각 셀을 부여해 실행하세요

In [ ]:
#!pip install opencv-python
#!pip install djitellopy
#!pip3 install keras
#!pip3 install tensorflow

##cv2 버전 4.8.1

In [ ]:
import cv2
print(cv2.__version__)

In [ ]:
import cv2
from djitellopy import Tello
from keras.models import load_model
import numpy as np

# YOLO 모델 및 설정 파일 경로 지정
#yolo 사용할 .cfg 파일과 .weights 파일을 같게하세요 두가지 모델을 사용할 수 없습니다.
weights_path = 'yolov3-tiny.weights' #성능 최하 속도 최상
cfg_path = 'yolov3-tiny.cfg' #성능 최하 속도 최상
#weights_path = 'yolov3-spp.weights' //성능 최상 속도 최하
#cfg_path = 'yolov3-spp.cfg'//성능 최상 속도 최하
#weights_path = 'yolov3_320.weights' //성능 하 속도 상
#cfg_path = 'yolov3_320.cfg' //성능 하 속도 상
#weights_path = 'yolov3_416.weights'//성능 중 속도 중
#cfg_path = 'yolov3_416.cfg' //성능 중 속도 중
class_names_path = 'coco.names'

# 클래스 이름 불러오기
with open(class_names_path, 'r') as f:
    class_names = [line.strip() for line in f.readlines()]

# YOLO 모델 로드
net = cv2.dnn.readNet(weights_path, cfg_path)
layer_names = net.getLayerNames()
out_layers = net.getUnconnectedOutLayers().flatten()
output_layers = [layer_names[i - 1] for i in out_layers]

# Tello 드론 연결 및 초기화
tello = Tello()
tello.connect()
tello.streamon()

# 영상 캡처 및 객체 탐지 수행
while True:
    frame = tello.get_frame_read().frame
    frame_resized = cv2.resize(frame, (416, 416))

    # YOLO 객체 탐지 코드
    blob = cv2.dnn.blobFromImage(frame_resized, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layers)

    # 탐지된 객체의 경계 상자를 화면에 표시
    for detection in detections[0]:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x, center_y, w, h = (detection[0:4] * np.array([416, 416, 416, 416])).astype('int')
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            cv2.rectangle(frame_resized, (x, y), (x + w, y + h), (0, 255, 0), 2)
            label = str(class_names[class_id])
            cv2.putText(frame_resized, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Tello Object Detection", frame_resized)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

tello.streamoff()
cv2.destroyAllWindows()
